In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer
import re

In [ ]:
#stopword_list = set(stopwords.words('english'))

stopword_list = set(["i", "me", "my", "myself","we", "our", "ours", "ourselves","you", "your", "yours",
               "their", "they", "his", "her","she", "he", "a", "an", "and","is", "was", "are", "were", 
               "him", "himself", "has", "have", "it", "its", "the", "us"])

def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def remove_special_characters(text, remove_digits=True):
    pattern=r'[^a-zA-z0-9\s]'
    text=re.sub(pattern,'',text)
    return text

def remove_noise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_special_characters(text)
    return text

In [ ]:
#Load the Train, Test, Test_y data
x_data = pd.read_csv("../data/alldata.tsv",sep="\t")
x_train = pd.read_csv("../data/split_0/train.csv")
x_test = pd.read_csv("../data/split_0/test.csv")
test_y = pd.read_csv("../data/split_0/test_y.csv")

In [ ]:
#Remove the Noise
#x_data["review"] = x_data["review"].apply(remove_noise_text)

In [ ]:
#Remove stopwords
x_data["review"] = x_data["review"].apply(lambda x: " ".join(x for x in x.split() if x.lower() not in stopword_list))

In [ ]:
df = (pd.Series(' '.join(x_data['review']).split())).value_counts().reset_index()
df = df.rename(columns={'index':'word',0:'word_count'})

In [ ]:
#Remove less frequent words
least_frequent_words = set(df[df['word_count'] < 11]['word'].str.lower())
x_data["review"] = x_data["review"].apply(lambda x: " ".join(x for x in x.split() if x.lower() not in least_frequent_words))


In [ ]:
#cv=CountVectorizer(min_df=0.001,max_df=0.5,ngram_range=(1,3))
cv=CountVectorizer(ngram_range=(1,3))
X_data_sparse = cv.fit_transform(x_data['review'])

X_train_sparse = cv.transform(x_train['review'])
X_test_sparse = cv.transform(x_test['review'])

print (len(cv.vocabulary_))
print (X_test_sparse.shape)
print (X_train_sparse.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, confusion_matrix

logit = LogisticRegression(random_state=17,n_jobs=-1, solver='lbfgs')

y_train = x_train['sentiment']
y_test = test_y['sentiment']

logit.fit(X_train_sparse, y_train)

print ("AUC: {}".format(round(roc_auc_score(y_test, logit.predict(X_test_sparse)), 3)))

In [ ]:
df = (pd.Series(' '.join(x_data['review']).split())).value_counts().reset_index()
df = df.rename(columns={'index':'word',0:'word_count'})

In [ ]:
least_frequent_words = set(df[df['word_count'] < 11]['word'].str.lower())
x_data["review"] = x_data["review"].apply(lambda x: " ".join(x for x in x.split() if x.lower() not in least_frequent_words))


In [ ]:
df1 = (pd.Series(' '.join(x_data['review']).split())).value_counts().reset_index()
df1 = df1.rename(columns={'index':'word',0:'word_count'})

In [ ]:
df1

In [ ]:
df1